In [44]:
%load_ext autoreload
%autoreload 2

from os import path, listdir, getcwd
from src.credentials import Credentials as cr
import csv
import requests
import sys
import os
import src.split_data as sd 
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:

sd.split_data(cr.TITLE_RATINGS)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\utilisateur\\Documents\\Intermovie\\Intermovie-Group1\\data\\RAW\\titles.ratings.tsv'